# Skyline Tests

This section contains some tests cases where the correctness of the implementation of the various skyline algorithms can be tested.

This works by executing each skyline query once for the "rewritten" equivalent skyline query in "pure" or "plain" SQL and then again for at least one skyline algorithm. The results of all queries are exported to .csv files where they can be compared.

It is advisible to sort all results by their respective unique identifiers to make comparing the results by hand easier.

**Known Potential Issue (WONTFIX)**: If there is a problem with the implementation of "plain" SQL queries than this problem is likely to propagate to both the skyline implementation and other regular queries. In this case, the skyline will yield wrong and potentially different results than the "plain" SQL query even if the implementation of the skyline itself is correct.

## List of Test Instances and Databses

In this section, we elaborate which tables and/or .csv files must exist or be loaded in Spark in order to execute the test cases properly. This serves for both configuration and documentation purposes.

**TODO:** Adapt variables and settings as needed.

In [ ]:
input_home = "/home/of/inputs/"       # terminate directory path by '/'

airbnb_test_source = "airbnb_test.csv"
fueleconomy_test_source = "fueleconomy_test.csv"
coil2000_test_source = "coil2000_test.csv"
nba_test_source = "nba_test.csv"

airbnb_test_source_incomplete = "airbnb_incomplete_test.csv"
fueleconomy_test_source_incomplete = "fueleconomy_incomplete_test.csv"
coil2000_test_source_incomplete = "coil2000_incomplete_test.csv"
nba_test_source_incomplete = "nba_incomplete_test.csv"

## Setup

This section is designated to setting up the test environment which includes finding the correct instance of Spark to be used in the test. It is highly advisible to use the same instance of Spark for all test cases and the "plain" reference query to ensure comparability.

**TODO:** Adapt variables and settings as needed.

In [ ]:
# %env SPARK_HOME=/home/lukas/spark/spark-3.1.2/
# %env PYTHONPATH=$SPARK_HOME/python:$PYTHONPATH
# %env PYTHONPATH=$SPARK_HOME/python/lib/py4j-0.10.9-src.zip:$PYTHONPATH
# %env PYSPARK_DRIVER_PYTHON="jupyter"
# %env PYSPARK_DRIVER_PYTHON_OPTS="notebook"
# %env PYSPARK_PYTHON=python3
# %env PATH=$SPARK_HOME:$PATH

airbnb_dims = ["price", "accommodates", "bedrooms", "beds", "number_of_reviews", "review_scores_rating"]
fueleconomy_dims = ["fuelCost08", "barrels08", "city08", "highway08", "comb08", "combinedCD"]
coil2000_dims = ["MOSHOOFD", "MGODRK", "MGODPR", "MGODGE", "MRELGE", "MRELOV"]
nba_dims = ["W", "L", "W_PCT", "FGM", "FGA", "FG_PCT"]

Check the environment:

In [ ]:
%env

Load Spark Context into ``sc`` and the SQL context into ``sqlContext``.

In [ ]:
import pyspark
from pyspark.sql.functions import *
from pyspark.sql import SQLContext

from pyspark.conf import SparkConf
from pyspark.context import SparkContext

conf = SparkConf()
conf.setMaster("spark://url:7077").setAll([('spark.sql.catalogImplementation', 'hive'), ('spark.executor.memory', '2g')])

sc = SparkContext(conf=conf)
sqlContext = SQLContext(sc)

Set up all input .csv files as tables to be used in query strings.

In [ ]:
sqlContext.sql("CREATE DATABASE IF NOT EXISTS tests")

df_airbnb = sqlContext.read.format('csv').options(header='true', inferschema='true').load(input_home + airbnb_test_source)
df_airbnb.registerTempTable("airbnb_test")
sqlContext.sql("CREATE TABLE IF NOT EXISTS tests.airbnb_test AS SELECT * FROM airbnb_test")

df_fueleconomy = sqlContext.read.format('csv').options(header='true', inferschema='true').load(input_home + fueleconomy_test_source)
df_fueleconomy.registerTempTable("fueleconomy_test")
sqlContext.sql("CREATE TABLE IF NOT EXISTS tests.fueleconomy_test AS SELECT * FROM fueleconomy_test")

df_coil2000 = sqlContext.read.format('csv').options(header='true', inferschema='true').load(input_home + coil2000_test_source)
df_coil2000.registerTempTable("coil2000_test")
sqlContext.sql("CREATE TABLE IF NOT EXISTS tests.coil2000_test AS SELECT * FROM coil2000_test")

df_nba = sqlContext.read.format('csv').options(header='true', inferschema='true').load(input_home + nba_test_source)
df_nba.registerTempTable("nba_test")
sqlContext.sql("CREATE TABLE IF NOT EXISTS tests.nba_test AS SELECT * FROM nba_test")

df_airbnb_incomplete = sqlContext.read.format('csv').options(header='true', inferschema='true').load(input_home + airbnb_test_source_incomplete)
df_airbnb_incomplete.registerTempTable("airbnb_test_incomplete")
sqlContext.sql("CREATE TABLE IF NOT EXISTS tests.airbnb_test_incomplete AS SELECT * FROM airbnb_test_incomplete")

df_fueleconomy_incomplete = sqlContext.read.format('csv').options(header='true', inferschema='true').load(input_home + fueleconomy_test_source_incomplete)
df_fueleconomy_incomplete.registerTempTable("fueleconomy_test_incomplete")
sqlContext.sql("CREATE TABLE IF NOT EXISTS tests.fueleconomy_test_incomplete AS SELECT * FROM fueleconomy_test_incomplete")

df_coil2000_incomplete = sqlContext.read.format('csv').options(header='true', inferschema='true').load(input_home + coil2000_test_source_incomplete)
df_coil2000_incomplete.registerTempTable("coil2000_test_incomplete")
sqlContext.sql("CREATE TABLE IF NOT EXISTS tests.coil2000_test_incomplete AS SELECT * FROM coil2000_test_incomplete")

df_nba_incomplete = sqlContext.read.format('csv').options(header='true', inferschema='true').load(input_home + nba_test_source_incomplete)
df_nba_incomplete.registerTempTable("nba_test_incomplete")
sqlContext.sql("CREATE TABLE IF NOT EXISTS tests.nba_test_incomplete AS SELECT * FROM nba_test_incomplete")

## Reference Solutions in Plain SQL

This section contains the reference queries for all test instances. Result files will be named after the dataset or table it is being executed on followed by the execution time in seconds.

### Complete Dataset References

In [ ]:
df_airbnb_reference_1d = sqlContext.sql("""
SELECT * FROM tests.airbnb_test AS o WHERE NOT EXISTS(
  SELECT * FROM tests.airbnb_test AS i WHERE
    i.price <= o.price
    AND (
      i.price < o.price
    )
) ORDER BY id
""")
df_airbnb_reference_1d.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_airbnb_reference_1d")

df_airbnb_reference_1d_distinct = df_airbnb_reference_1d.dropDuplicates(airbnb_dims[:1]).orderBy("id")
df_airbnb_reference_1d_distinct.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_airbnb_reference_1d_distinct")

df_airbnb_reference_2d = sqlContext.sql("""
SELECT * FROM tests.airbnb_test AS o WHERE NOT EXISTS(
  SELECT * FROM tests.airbnb_test AS i WHERE
    i.price <= o.price AND
    i.accommodates >= o.accommodates
    AND (
      i.price < o.price OR
      i.accommodates > o.accommodates
    )
) ORDER BY id
""")
df_airbnb_reference_2d.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_airbnb_reference_2d")

df_airbnb_reference_2d_distinct = df_airbnb_reference_2d.dropDuplicates(airbnb_dims[:2]).orderBy("id")
df_airbnb_reference_2d_distinct.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_airbnb_reference_2d_distinct")

df_airbnb_reference_6d = sqlContext.sql("""
SELECT * FROM tests.airbnb_test AS o WHERE NOT EXISTS(
  SELECT * FROM tests.airbnb_test AS i WHERE
    i.price <= o.price AND
    i.accommodates >= o.accommodates AND
    i.bedrooms >= o.bedrooms AND
    i.beds >= o.beds AND
    i.number_of_reviews >= o.number_of_reviews AND
    i.review_scores_rating >= o.review_scores_rating
    AND (
      i.price < o.price OR
      i.accommodates > o.accommodates OR
      i.bedrooms > o.bedrooms OR
      i.beds > o.beds OR
      i.number_of_reviews > o.number_of_reviews OR
      i.review_scores_rating > o.review_scores_rating
    )
) ORDER BY id
""")
df_airbnb_reference_6d.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_airbnb_reference_6d")

df_airbnb_reference_6d_distinct = df_airbnb_reference_6d.dropDuplicates(airbnb_dims[:6]).orderBy("id")
df_airbnb_reference_6d_distinct.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_airbnb_reference_6d_distinct")


In [ ]:
df_fueleconomy_reference_1d = sqlContext.sql("""
SELECT * FROM tests.fueleconomy_test AS o WHERE NOT EXISTS(
    SELECT * FROM tests.fueleconomy_test AS i WHERE
        i.fuelCost08 <= o.fuelCost08
        AND (
          i.fuelCost08 < o.fuelCost08
        )
) ORDER BY make, model
""")
df_fueleconomy_reference_1d.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_fueleconomy_reference_1d")

df_fueleconomy_reference_1d_distinct = df_fueleconomy_reference_1d.dropDuplicates(fueleconomy_dims[:1]).orderBy("make", "model")
df_fueleconomy_reference_1d_distinct.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_fueleconomy_reference_1d_distinct")

df_fueleconomy_reference_2d = sqlContext.sql("""
SELECT * FROM tests.fueleconomy_test AS o WHERE NOT EXISTS(
  SELECT * FROM tests.fueleconomy_test AS i WHERE
    i.fuelCost08 <= o.fuelCost08 AND
    i.barrels08 <= o.barrels08
    AND (
      i.fuelCost08 < o.fuelCost08 OR
      i.barrels08 < o.barrels08
    )
) ORDER BY make, model
""")
df_fueleconomy_reference_2d.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_fueleconomy_reference_2d")

df_fueleconomy_reference_2d_distinct = df_fueleconomy_reference_2d.dropDuplicates(fueleconomy_dims[:2]).orderBy("make", "model")
df_fueleconomy_reference_2d_distinct.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_fueleconomy_reference_2d_distinct")

df_fueleconomy_reference_6d = sqlContext.sql("""
SELECT * FROM tests.fueleconomy_test AS o WHERE NOT EXISTS(
  SELECT * FROM tests.fueleconomy_test AS i WHERE
    i.fuelCost08 <= o.fuelCost08 AND
    i.barrels08 <= o.barrels08 AND
    i.city08 >= o.city08 AND
    i.highway08 >= o.highway08 AND
    i.comb08 >= o.comb08 AND
    i.combinedCD <= o.combinedCD
    AND (
      i.fuelCost08 < o.fuelCost08 OR
      i.barrels08 < o.barrels08 OR
      i.city08 > o.city08 OR
      i.highway08 > o.highway08 OR
      i.comb08 > o.comb08 OR
      i.combinedCD < o.combinedCD
    )
) ORDER BY make, model
""")
df_fueleconomy_reference_6d.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_fueleconomy_reference_6d")

df_fueleconomy_reference_6d_distinct = df_fueleconomy_reference_6d.dropDuplicates(fueleconomy_dims[:6]).orderBy("make", "model")
df_fueleconomy_reference_6d_distinct.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_fueleconomy_reference_6d_distinct")


In [ ]:
df_coil2000_reference_1d = sqlContext.sql("""
SELECT * FROM tests.coil2000_test AS o WHERE NOT EXISTS(
  SELECT * FROM tests.coil2000_test AS i WHERE
    i.MOSHOOFD >= o.MOSHOOFD
    AND (
      i.MOSHOOFD > o.MOSHOOFD
    )
) ORDER BY id
""")
df_coil2000_reference_1d.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_coil2000_reference_1d")

df_coil2000_reference_1d_distinct = df_coil2000_reference_1d.dropDuplicates(coil2000_dims[:1]).orderBy("id")
df_coil2000_reference_1d_distinct.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_coil2000_reference_1d_distinct")

df_coil2000_reference_2d = sqlContext.sql("""
SELECT * FROM tests.coil2000_test AS o WHERE NOT EXISTS(
  SELECT * FROM tests.coil2000_test AS i WHERE
    i.MOSHOOFD >= o.MOSHOOFD AND
    i.MGODRK <= o.MGODRK
    AND (
      i.MOSHOOFD > o.MOSHOOFD OR
      i.MGODRK < o.MGODRK
    )
) ORDER BY id
""")
df_coil2000_reference_2d.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_coil2000_reference_2d")

df_coil2000_reference_2d_distinct = df_coil2000_reference_2d.dropDuplicates(coil2000_dims[:2]).orderBy("id")
df_coil2000_reference_2d_distinct.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_coil2000_reference_2d_distinct")

df_coil2000_reference_6d = sqlContext.sql("""
SELECT * FROM tests.coil2000_test AS o WHERE NOT EXISTS(
  SELECT * FROM tests.coil2000_test AS i WHERE
    i.MOSHOOFD >= o.MOSHOOFD AND
    i.MGODRK <= o.MGODRK AND
    i.MGODPR <= o.MGODPR AND
    i.MGODGE >= o.MGODGE AND
    i.MRELGE >= o.MRELGE AND
    i.MRELOV >= o.MRELOV
    AND (
      i.MOSHOOFD > o.MOSHOOFD OR
      i.MGODRK < o.MGODRK OR
      i.MGODPR < o.MGODPR OR
      i.MGODGE > o.MGODGE OR
      i.MRELGE > o.MRELGE OR
      i.MRELOV > o.MRELOV
    )
) ORDER BY id
""")
df_coil2000_reference_6d.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_coil2000_reference_6d")

df_coil2000_reference_6d_distinct = df_coil2000_reference_6d.dropDuplicates(coil2000_dims[:6]).orderBy("id")
df_coil2000_reference_6d_distinct.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_coil2000_reference_6d_distinct")

In [ ]:
df_nba_reference_1d = sqlContext.sql("""
SELECT * FROM tests.nba_test AS o WHERE NOT EXISTS(
  SELECT * FROM tests.nba_test AS i WHERE
    i.W >= o.W
    AND (
      i.W > o.W
    )
) ORDER BY PLAYER_ID
""")
df_nba_reference_1d.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_nba_reference_1d")

df_nba_reference_1d_distinct = df_nba_reference_1d.dropDuplicates(nba_dims[:1]).orderBy("PLAYER_ID")
df_nba_reference_1d_distinct.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_nba_reference_1d_distinct")

df_nba_reference_2d = sqlContext.sql("""
SELECT * FROM tests.nba_test AS o WHERE NOT EXISTS(
  SELECT * FROM tests.nba_test AS i WHERE
    i.W >= o.W AND
    i.L <= o.L
    AND (
      i.W > o.W OR
      i.L < o.L
    )
) ORDER BY PLAYER_ID
""")
df_nba_reference_2d.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_nba_reference_2d")

df_nba_reference_2d_distinct = df_nba_reference_2d.dropDuplicates(nba_dims[:2]).orderBy("PLAYER_ID")
df_nba_reference_2d_distinct.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_nba_reference_2d_distinct")

df_nba_reference_6d = sqlContext.sql("""
SELECT * FROM tests.nba_test AS o WHERE NOT EXISTS(
  SELECT * FROM tests.nba_test AS i WHERE
    i.W >= o.W AND
    i.L <= o.L AND
    i.W_PCT >= o.W_PCT AND
    i.FGM >= o.FGM AND
    i.FGA >= o.FGA AND
    i.FG_PCT >= o.FG_PCT
    AND (
      i.W > o.W OR
      i.L < o.L OR
      i.W_PCT > o.W_PCT OR
      i.FGM > o.FGM OR
      i.FGA > o.FGA OR
      i.FG_PCT > o.FG_PCT
    )
) ORDER BY PLAYER_ID
""")
df_nba_reference_6d.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_nba_reference_6d")

df_nba_reference_6d_distinct = df_nba_reference_6d.dropDuplicates(nba_dims[:6]).orderBy("PLAYER_ID")
df_nba_reference_6d_distinct.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_nba_reference_6d_distinct")


### Incomplete Dataset References

In [ ]:
df_airbnb_incomplete_reference_1d = sqlContext.sql("""
SELECT * FROM tests.airbnb_test_incomplete AS o WHERE NOT EXISTS(
  SELECT * FROM tests.airbnb_test_incomplete AS i WHERE
    (i.price <= o.price OR i.price IS NULL OR o.price IS NULL)
    AND (
      (i.price < o.price AND i.price IS NOT NULL AND o.price IS NOT NULL)
    )
) ORDER BY id
""")
df_airbnb_incomplete_reference_1d.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_airbnb_incomplete_reference_1d")

df_airbnb_incomplete_reference_1d_distinct = df_airbnb_incomplete_reference_1d.dropDuplicates(airbnb_dims[:1]).orderBy("id")
df_airbnb_incomplete_reference_1d_distinct.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_airbnb_incomplete_reference_1d_distinct")

df_airbnb_incomplete_reference_2d = sqlContext.sql("""
SELECT * FROM tests.airbnb_test_incomplete AS o WHERE NOT EXISTS(
  SELECT * FROM tests.airbnb_test_incomplete AS i WHERE
    (i.price <= o.price OR i.price IS NULL OR o.price IS NULL) AND
    (i.accommodates >= o.accommodates OR i.accommodates IS NULL OR o.accommodates IS NULL)
    AND (
      (i.price < o.price AND i.price IS NOT NULL AND o.price IS NOT NULL) OR
      (i.accommodates > o.accommodates AND i.accommodates IS NOT NULL AND o.accommodates IS NOT NULL)
    )
) ORDER BY id
""")
df_airbnb_incomplete_reference_2d.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_airbnb_incomplete_reference_2d")

df_airbnb_incomplete_reference_2d_distinct = df_airbnb_incomplete_reference_2d.dropDuplicates(airbnb_dims[:2]).orderBy("id")
df_airbnb_incomplete_reference_2d_distinct.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_airbnb_incomplete_reference_2d_distinct")

df_airbnb_incomplete_reference_6d = sqlContext.sql("""
SELECT * FROM tests.airbnb_test_incomplete AS o WHERE NOT EXISTS(
  SELECT * FROM tests.airbnb_test_incomplete AS i WHERE
    (i.price <= o.price OR i.price IS NULL OR o.price IS NULL) AND
    (i.accommodates >= o.accommodates OR i.accommodates IS NULL OR o.accommodates IS NULL) AND
    (i.bedrooms >= o.bedrooms OR i.bedrooms IS NULL OR o.bedrooms IS NULL) AND
    (i.beds >= o.beds OR i.beds IS NULL OR o.beds IS NULL) AND
    (i.number_of_reviews >= o.number_of_reviews OR i.number_of_reviews IS NULL OR o.number_of_reviews IS NULL) AND
    (i.review_scores_rating >= o.review_scores_rating OR i.review_scores_rating IS NULL OR o.review_scores_rating IS NULL)
    AND (
      (i.price < o.price AND i.price IS NOT NULL AND o.price IS NOT NULL) OR
      (i.accommodates > o.accommodates AND i.accommodates IS NOT NULL AND o.accommodates IS NOT NULL) OR
      (i.bedrooms > o.bedrooms AND i.bedrooms IS NOT NULL AND o.bedrooms IS NOT NULL) OR
      (i.beds > o.beds AND i.beds IS NOT NULL AND o.beds IS NOT NULL) OR
      (i.number_of_reviews > o.number_of_reviews AND i.number_of_reviews IS NOT NULL AND o.number_of_reviews IS NOT NULL) OR
      (i.review_scores_rating > o.review_scores_rating AND i.review_scores_rating IS NOT NULL AND o.review_scores_rating IS NOT NULL)
    )
) ORDER BY id
""")
df_airbnb_incomplete_reference_6d.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_airbnb_incomplete_reference_6d")

df_airbnb_incomplete_reference_6d_distinct = df_airbnb_incomplete_reference_6d.dropDuplicates(airbnb_dims[:6]).orderBy("id")
df_airbnb_incomplete_reference_6d_distinct.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_airbnb_incomplete_reference_6d_distinct")

In [ ]:
df_fueleconomy_incomplete_reference_1d = sqlContext.sql("""
SELECT * FROM tests.fueleconomy_test_incomplete AS o WHERE NOT EXISTS(
  SELECT * FROM tests.fueleconomy_test_incomplete AS i WHERE
    (i.fuelCost08 <= o.fuelCost08 OR i.fuelCost08 IS NULL OR o.fuelCost08 IS NULL)
    AND (
      (i.fuelCost08 < o.fuelCost08 AND i.fuelCost08 IS NOT NULL AND o.fuelCost08 IS NOT NULL)
    )
) ORDER BY make, model
""")
df_fueleconomy_incomplete_reference_1d.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_fueleconomy_incomplete_reference_1d")

df_fueleconomy_incomplete_reference_1d_distinct = df_fueleconomy_incomplete_reference_1d.dropDuplicates(fueleconomy_dims[:1]).orderBy("make", "model")
df_fueleconomy_incomplete_reference_1d_distinct.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_fueleconomy_incomplete_reference_1d_distinct")

df_fueleconomy_incomplete_reference_2d = sqlContext.sql("""
SELECT * FROM tests.fueleconomy_test_incomplete AS o WHERE NOT EXISTS(
  SELECT * FROM tests.fueleconomy_test_incomplete AS i WHERE
    (i.fuelCost08 <= o.fuelCost08 OR i.fuelCost08 IS NULL OR o.fuelCost08 IS NULL) AND
    (i.barrels08 <= o.barrels08 OR i.barrels08 IS NULL OR o.barrels08 IS NULL)
    AND (
      (i.fuelCost08 < o.fuelCost08 AND i.fuelCost08 IS NOT NULL AND o.fuelCost08 IS NOT NULL) OR
      (i.barrels08 < o.barrels08 AND i.barrels08 IS NOT NULL AND o.barrels08 IS NOT NULL)
    )
) ORDER BY make, model
""")
df_fueleconomy_incomplete_reference_2d.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_fueleconomy_incomplete_reference_2d")

df_fueleconomy_incomplete_reference_2d_distinct = df_fueleconomy_incomplete_reference_2d.dropDuplicates(fueleconomy_dims[:2]).orderBy("make", "model")
df_fueleconomy_incomplete_reference_2d_distinct.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_fueleconomy_incomplete_reference_2d_distinct")


df_fueleconomy_incomplete_reference_6d = sqlContext.sql("""
SELECT * FROM tests.fueleconomy_test_incomplete AS o WHERE NOT EXISTS(
  SELECT * FROM tests.fueleconomy_test_incomplete AS i WHERE
    (i.fuelCost08 <= o.fuelCost08 OR i.fuelCost08 IS NULL OR o.fuelCost08 IS NULL) AND
    (i.barrels08 <= o.barrels08 OR i.barrels08 IS NULL OR o.barrels08 IS NULL) AND
    (i.city08 >= o.city08 OR i.city08 IS NULL OR o.city08 IS NULL) AND
    (i.highway08 >= o.highway08 OR i.highway08 IS NULL OR o.highway08 IS NULL) AND
    (i.comb08 >= o.comb08 OR i.comb08 IS NULL OR o.comb08 IS NULL) AND
    (i.combinedCD <= o.combinedCD OR i.combinedCD IS NULL OR o.combinedCD IS NULL)
    AND (
      (i.fuelCost08 < o.fuelCost08 AND i.fuelCost08 IS NOT NULL AND o.fuelCost08 IS NOT NULL) OR
      (i.barrels08 < o.barrels08 AND i.barrels08 IS NOT NULL AND o.barrels08 IS NOT NULL) OR
      (i.city08 > o.city08 AND i.city08 IS NOT NULL AND o.city08 IS NOT NULL) OR
      (i.highway08 > o.highway08 AND i.highway08 IS NOT NULL AND o.highway08 IS NOT NULL) OR
      (i.comb08 > o.comb08 AND i.comb08 IS NOT NULL AND o.comb08 IS NOT NULL) OR
      (i.combinedCD < o.combinedCD AND i.combinedCD IS NOT NULL AND o.combinedCD IS NOT NULL)
    )
) ORDER BY make, model
""")
df_fueleconomy_incomplete_reference_6d.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_fueleconomy_incomplete_reference_6d")

df_fueleconomy_incomplete_reference_6d_distinct = df_fueleconomy_incomplete_reference_6d.dropDuplicates(fueleconomy_dims[:6]).orderBy("make", "model")
df_fueleconomy_incomplete_reference_6d_distinct.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_fueleconomy_incomplete_reference_6d_distinct")


In [ ]:
df_coil2000_incomplete_reference_1d = sqlContext.sql("""
SELECT * FROM tests.coil2000_test_incomplete AS o WHERE NOT EXISTS(
  SELECT * FROM tests.coil2000_test_incomplete AS i WHERE
    (i.MOSHOOFD >= o.MOSHOOFD OR i.MOSHOOFD IS NULL OR o.MOSHOOFD IS NULL)
    AND (
      (i.MOSHOOFD > o.MOSHOOFD AND i.MOSHOOFD IS NOT NULL AND o.MOSHOOFD IS NOT NULL)
    )
) ORDER BY id
""")
df_coil2000_incomplete_reference_1d.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_coil2000_incomplete_reference_1d")

df_coil2000_incomplete_reference_1d_distinct = df_coil2000_incomplete_reference_1d.dropDuplicates(coil2000_dims[:1]).orderBy("id")
df_coil2000_incomplete_reference_1d_distinct.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_coil2000_incomplete_reference_1d_distinct")

df_coil2000_incomplete_reference_2d = sqlContext.sql("""
SELECT * FROM tests.coil2000_test_incomplete AS o WHERE NOT EXISTS(
  SELECT * FROM tests.coil2000_test_incomplete AS i WHERE
    (i.MOSHOOFD >= o.MOSHOOFD OR i.MOSHOOFD IS NULL OR o.MOSHOOFD IS NULL) AND
    (i.MGODRK <= o.MGODRK OR i.MGODRK IS NULL OR o.MGODRK IS NULL)
    AND (
      (i.MOSHOOFD > o.MOSHOOFD AND i.MOSHOOFD IS NOT NULL AND o.MOSHOOFD IS NOT NULL) OR
      (i.MGODRK < o.MGODRK AND i.MGODRK IS NOT NULL AND o.MGODRK IS NOT NULL)
    )
) ORDER BY id
""")
df_coil2000_incomplete_reference_2d.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_coil2000_incomplete_reference_2d")

df_coil2000_incomplete_reference_2d_distinct = df_coil2000_incomplete_reference_2d.dropDuplicates(coil2000_dims[:2]).orderBy("id")
df_coil2000_incomplete_reference_2d_distinct.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_coil2000_incomplete_reference_2d_distinct")

df_coil2000_incomplete_reference_6d = sqlContext.sql("""
SELECT * FROM tests.coil2000_test_incomplete AS o WHERE NOT EXISTS(
  SELECT * FROM tests.coil2000_test_incomplete AS i WHERE
    (i.MOSHOOFD >= o.MOSHOOFD OR i.MOSHOOFD IS NULL OR o.MOSHOOFD IS NULL) AND
    (i.MGODRK <= o.MGODRK OR i.MGODRK IS NULL OR o.MGODRK IS NULL) AND
    (i.MGODPR <= o.MGODPR OR i.MGODPR IS NULL OR o.MGODPR IS NULL) AND
    (i.MGODGE >= o.MGODGE OR i.MGODGE IS NULL OR o.MGODGE IS NULL) AND
    (i.MRELGE >= o.MRELGE OR i.MRELGE IS NULL OR o.MRELGE IS NULL) AND
    (i.MRELOV >= o.MRELOV OR i.MRELOV IS NULL OR o.MRELOV IS NULL)
    AND (
      (i.MOSHOOFD > o.MOSHOOFD AND i.MOSHOOFD IS NOT NULL AND o.MOSHOOFD IS NOT NULL) OR
      (i.MGODRK < o.MGODRK AND i.MGODRK IS NOT NULL AND o.MGODRK IS NOT NULL) OR 
      (i.MGODPR < o.MGODPR AND i.MGODPR IS NOT NULL AND o.MGODPR IS NOT NULL) OR
      (i.MGODGE > o.MGODGE AND i.MGODGE IS NOT NULL AND o.MGODGE IS NOT NULL) OR
      (i.MRELGE > o.MRELGE AND i.MRELGE IS NOT NULL AND o.MRELGE IS NOT NULL) OR
      (i.MRELOV > o.MRELOV AND i.MRELOV IS NOT NULL AND o.MRELOV IS NOT NULL)
    )
) ORDER BY id
""")
df_coil2000_incomplete_reference_6d.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_coil2000_incomplete_reference_6d")

df_coil2000_incomplete_reference_6d_distinct = df_coil2000_incomplete_reference_6d.dropDuplicates(coil2000_dims[:6]).orderBy("id")
df_coil2000_incomplete_reference_6d_distinct.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_coil2000_incomplete_reference_6d_distinct")

In [ ]:
df_nba_incomplete_reference_1d = sqlContext.sql("""
SELECT * FROM tests.nba_test_incomplete AS o WHERE NOT EXISTS(
  SELECT * FROM tests.nba_test_incomplete AS i WHERE
    (i.W >= o.W OR i.W IS NULL OR o.W IS NULL)
    AND (
      (i.W > o.W AND i.W IS NOT NULL AND o.W IS NOT NULL)
    )
) ORDER BY PLAYER_ID
""")
df_nba_incomplete_reference_1d.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_nba_incomplete_reference_1d")

df_nba_incomplete_reference_1d_distinct = df_nba_incomplete_reference_1d.dropDuplicates(nba_dims[:1]).orderBy("PLAYER_ID")
df_nba_incomplete_reference_1d_distinct.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_nba_incomplete_reference_1d_distinct")

df_nba_incomplete_reference_2d = sqlContext.sql("""
SELECT * FROM tests.nba_test_incomplete AS o WHERE NOT EXISTS(
  SELECT * FROM tests.nba_test_incomplete AS i WHERE
    (i.W >= o.W OR i.W IS NULL OR o.W IS NULL) AND
    (i.L <= o.L OR i.L IS NULL OR o.L IS NULL)
    AND (
      (i.W > o.W AND i.W IS NOT NULL AND o.W IS NOT NULL) OR
      (i.L < o.L AND i.L IS NOT NULL AND o.L IS NOT NULL)
    )
) ORDER BY PLAYER_ID
""")
df_nba_incomplete_reference_2d.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_nba_incomplete_reference_2d")

df_nba_incomplete_reference_2d_distinct = df_nba_incomplete_reference_2d.dropDuplicates(nba_dims[:2]).orderBy("PLAYER_ID")
df_nba_incomplete_reference_2d_distinct.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_nba_incomplete_reference_2d_distinct")

df_nba_incomplete_reference_6d = sqlContext.sql("""
SELECT * FROM tests.nba_test_incomplete AS o WHERE NOT EXISTS(
  SELECT * FROM tests.nba_test_incomplete AS i WHERE
    (i.W >= o.W OR i.W IS NULL OR o.W IS NULL) AND
    (i.L <= o.L OR i.L IS NULL OR o.L IS NULL) AND
    (i.W_PCT >= o.W_PCT OR i.W_PCT IS NULL OR o.W_PCT IS NULL) AND
    (i.FGM >= o.FGM OR i.FGM IS NULL OR o.FGM IS NULL) AND
    (i.FGA >= o.FGA OR i.FGA IS NULL OR o.FGA IS NULL) AND
    (i.FG_PCT >= o.FG_PCT OR i.FG_PCT IS NULL OR o.FG_PCT IS NULL)
    AND (
      (i.W > o.W AND i.W IS NOT NULL AND o.W IS NOT NULL) OR
      (i.L < o.L AND i.L IS NOT NULL AND o.L IS NOT NULL) OR
      (i.W_PCT > o.W_PCT AND i.W_PCT IS NOT NULL AND o.W_PCT IS NOT NULL) OR
      (i.FGM > o.FGM AND i.FGM IS NOT NULL AND o.FGM IS NOT NULL) OR
      (i.FGA > o.FGA AND i.FGA IS NOT NULL AND o.FGA IS NOT NULL) OR
      (i.FG_PCT > o.FG_PCT AND i.FG_PCT IS NOT NULL AND o.FG_PCT IS NOT NULL)
    )
) ORDER BY PLAYER_ID
""")
df_nba_incomplete_reference_6d.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_nba_incomplete_reference_6d")

df_nba_incomplete_reference_6d_distinct = df_nba_incomplete_reference_6d.dropDuplicates(nba_dims[:6]).orderBy("PLAYER_ID")
df_nba_incomplete_reference_6d_distinct.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_nba_incomplete_reference_6d_distinct")

## Skyline Solutions Block-Nested-Loop

This section contains the results of using the block-nested-loop algorithm to compute the skyline.

### Non-Distinct Queries

In this subsection we calculate the results for each non-distinct skyline query. For distinct skyline queries see the next subsection.

In [ ]:
df_airbnb_bnl_1d = sqlContext.sql("""
SELECT * FROM tests.airbnb_test SKYLINE OF BNL
  price MIN
ORDER BY id
""")
df_airbnb_bnl_1d.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_airbnb_bnl_1d")

df_airbnb_bnl_2d = sqlContext.sql("""
SELECT * FROM tests.airbnb_test SKYLINE OF BNL
  price MIN,
  accommodates MAX
ORDER BY id
""")
df_airbnb_bnl_2d.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_airbnb_bnl_2d")

df_airbnb_bnl_6d = sqlContext.sql("""
SELECT * FROM tests.airbnb_test SKYLINE OF BNL
  price MIN,
  accommodates MAX,
  bedrooms MAX,
  beds MAX,
  number_of_reviews MAX,
  review_scores_rating MAX
ORDER BY id
""")
df_airbnb_bnl_6d.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_airbnb_bnl_6d")

In [ ]:
df_fueleconomy_bnl_1d = sqlContext.sql("""
SELECT * FROM tests.fueleconomy_test SKYLINE OF BNL
  fuelCost08 MIN
ORDER BY make, model
""")
df_fueleconomy_bnl_1d.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_fueleconomy_bnl_1d")

df_fueleconomy_bnl_2d = sqlContext.sql("""
SELECT * FROM tests.fueleconomy_test SKYLINE OF BNL
  fuelCost08 MIN,
  barrels08 MIN
ORDER BY make, model
""")
df_fueleconomy_bnl_2d.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_fueleconomy_bnl_2d")

df_fueleconomy_bnl_6d = sqlContext.sql("""
SELECT * FROM tests.fueleconomy_test SKYLINE OF BNL
  fuelCost08 MIN,
  barrels08 MIN,
  city08 MAX,
  highway08 MAX,
  comb08 MAX,
  combinedCD MIN
ORDER BY make, model
""")
df_fueleconomy_bnl_6d.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_fueleconomy_bnl_6d")

In [ ]:
df_coil2000_bnl_1d = sqlContext.sql("""
SELECT * FROM tests.coil2000_test SKYLINE OF BNL
  MOSHOOFD MAX
ORDER BY id
""")
df_coil2000_bnl_1d.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_coil2000_bnl_1d")

df_coil2000_bnl_2d = sqlContext.sql("""
SELECT * FROM tests.coil2000_test SKYLINE OF BNL
  MOSHOOFD MAX,
  MGODRK MIN
ORDER BY id
""")
df_coil2000_bnl_2d.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_coil2000_bnl_2d")

df_coil2000_bnl_6d = sqlContext.sql("""
SELECT * FROM tests.coil2000_test SKYLINE OF BNL
  MOSHOOFD MAX,
  MGODRK MIN,
  MGODPR MIN,
  MGODGE MAX,
  MRELGE MAX,
  MRELOV MAX
ORDER BY id
""")
df_coil2000_bnl_6d.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_coil2000_bnl_6d")

In [ ]:
df_nba_bnl_1d = sqlContext.sql("""
SELECT * FROM tests.nba_test SKYLINE OF BNL
  W MAX
ORDER BY PLAYER_ID
""")
df_nba_bnl_1d.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_nba_bnl_1d")

df_nba_bnl_2d = sqlContext.sql("""
SELECT * FROM tests.nba_test SKYLINE OF BNL
  W MAX,
  L MIN
ORDER BY PLAYER_ID
""")
df_nba_bnl_2d.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_nba_bnl_2d")

df_nba_bnl_6d = sqlContext.sql("""
SELECT * FROM tests.nba_test SKYLINE OF BNL
  W MAX,
  L MIN,
  W_PCT MAX,
  FGM MAX,
  FGA MAX,
  FG_PCT MAX
ORDER BY PLAYER_ID
""")
df_nba_bnl_6d.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_nba_bnl_6d")

### Distinct Queries

In this subsection we calculate the results for each distinct skyline query. For non-distinct skyline queries see the previous subsection.

In [ ]:
df_airbnb_bnl_1d_distinct = sqlContext.sql("""
SELECT * FROM tests.airbnb_test SKYLINE OF DISTINCT BNL
  price MIN
ORDER BY id
""")
df_airbnb_bnl_1d_distinct.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_airbnb_bnl_1d_distinct")

df_airbnb_bnl_2d_distinct = sqlContext.sql("""
SELECT * FROM tests.airbnb_test SKYLINE OF DISTINCT BNL
  price MIN,
  accommodates MAX
ORDER BY id
""")
df_airbnb_bnl_2d_distinct.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_airbnb_bnl_2d_distinct")

df_airbnb_bnl_6d_distinct = sqlContext.sql("""
SELECT * FROM tests.airbnb_test SKYLINE OF DISTINCT BNL
  price MIN,
  accommodates MAX,
  bedrooms MAX,
  beds MAX,
  number_of_reviews MAX,
  review_scores_rating MAX
ORDER BY id
""")
df_airbnb_bnl_6d_distinct.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_airbnb_bnl_6d_distinct")

In [ ]:
df_fueleconomy_bnl_1d_distinct = sqlContext.sql("""
SELECT * FROM tests.fueleconomy_test SKYLINE OF DISTINCT BNL
  fuelCost08 MIN
ORDER BY make, model
""")
df_fueleconomy_bnl_1d_distinct.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_fueleconomy_bnl_1d_distinct")

df_fueleconomy_bnl_2d_distinct = sqlContext.sql("""
SELECT * FROM tests.fueleconomy_test SKYLINE OF DISTINCT BNL
  fuelCost08 MIN,
  barrels08 MIN
ORDER BY make, model
""")
df_fueleconomy_bnl_2d_distinct.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_fueleconomy_bnl_2d_distinct")

df_fueleconomy_bnl_6d_distinct = sqlContext.sql("""
SELECT * FROM tests.fueleconomy_test SKYLINE OF DISTINCT BNL
  fuelCost08 MIN,
  barrels08 MIN,
  city08 MAX,
  highway08 MAX,
  comb08 MAX,
  combinedCD MIN
ORDER BY make, model
""")
df_fueleconomy_bnl_6d_distinct.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_fueleconomy_bnl_6d_distinct")

In [ ]:
df_coil2000_bnl_1d_distinct = sqlContext.sql("""
SELECT * FROM tests.coil2000_test SKYLINE OF DISTINCT BNL
  MOSHOOFD MAX
ORDER BY id
""")
df_coil2000_bnl_1d_distinct.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_coil2000_bnl_1d_distinct")

df_coil2000_bnl_2d_distinct = sqlContext.sql("""
SELECT * FROM tests.coil2000_test SKYLINE OF DISTINCT BNL
  MOSHOOFD MAX,
  MGODRK MIN
ORDER BY id
""")
df_coil2000_bnl_2d_distinct.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_coil2000_bnl_2d_distinct")

df_coil2000_bnl_6d_distinct = sqlContext.sql("""
SELECT * FROM tests.coil2000_test SKYLINE OF DISTINCT BNL
  MOSHOOFD MAX,
  MGODRK MIN,
  MGODPR MIN,
  MGODGE MAX,
  MRELGE MAX,
  MRELOV MAX
ORDER BY id
""")
df_coil2000_bnl_6d_distinct.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_coil2000_bnl_6d_distinct")

In [ ]:
df_nba_bnl_1d_distinct = sqlContext.sql("""
SELECT * FROM tests.nba_test SKYLINE OF DISTINCT BNL
  W MAX
ORDER BY PLAYER_ID
""")
df_nba_bnl_1d_distinct.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_nba_bnl_1d_distinct")

df_nba_bnl_2d_distinct = sqlContext.sql("""
SELECT * FROM tests.nba_test SKYLINE OF DISTINCT BNL
  W MAX,
  L MIN
ORDER BY PLAYER_ID
""")
df_nba_bnl_2d_distinct.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_nba_bnl_2d_distinct")

df_nba_bnl_6d_distinct = sqlContext.sql("""
SELECT * FROM tests.nba_test SKYLINE OF DISTINCT BNL
  W MAX,
  L MIN,
  W_PCT MAX,
  FGM MAX,
  FGA MAX,
  FG_PCT MAX
ORDER BY PLAYER_ID
""")
df_nba_bnl_6d_distinct.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_nba_bnl_6d_distinct")

### Result Area

In [ ]:
display("non-distinct:")

display(len(df_airbnb_bnl_1d.exceptAll(df_airbnb_reference_1d).head(1)) == 0)
display(len(df_airbnb_reference_1d.exceptAll(df_airbnb_bnl_1d).head(1)) == 0)
display(len(df_airbnb_bnl_2d.exceptAll(df_airbnb_reference_2d).head(1)) == 0)
display(len(df_airbnb_reference_2d.exceptAll(df_airbnb_bnl_2d).head(1)) == 0)
display(len(df_airbnb_bnl_6d.exceptAll(df_airbnb_reference_6d).head(1)) == 0)
display(len(df_airbnb_reference_6d.exceptAll(df_airbnb_bnl_6d).head(1)) == 0)

display(len(df_fueleconomy_bnl_1d.exceptAll(df_fueleconomy_reference_1d).head(1)) == 0)
display(len(df_fueleconomy_reference_1d.exceptAll(df_fueleconomy_bnl_1d).head(1)) == 0)
display(len(df_fueleconomy_bnl_2d.exceptAll(df_fueleconomy_reference_2d).head(1)) == 0)
display(len(df_fueleconomy_reference_2d.exceptAll(df_fueleconomy_bnl_2d).head(1)) == 0)
display(len(df_fueleconomy_bnl_6d.exceptAll(df_fueleconomy_reference_6d).head(1)) == 0)
display(len(df_fueleconomy_reference_6d.exceptAll(df_fueleconomy_bnl_6d).head(1)) == 0)

display(len(df_coil2000_bnl_1d.exceptAll(df_coil2000_reference_1d).head(1)) == 0)
display(len(df_coil2000_reference_1d.exceptAll(df_coil2000_bnl_1d).head(1)) == 0)
display(len(df_coil2000_bnl_2d.exceptAll(df_coil2000_reference_2d).head(1)) == 0)
display(len(df_coil2000_reference_2d.exceptAll(df_coil2000_bnl_2d).head(1)) == 0)
display(len(df_coil2000_bnl_6d.exceptAll(df_coil2000_reference_6d).head(1)) == 0)
display(len(df_coil2000_reference_6d.exceptAll(df_coil2000_bnl_6d).head(1)) == 0)

display(len(df_nba_bnl_1d.exceptAll(df_nba_reference_1d).head(1)) == 0)
display(len(df_nba_reference_1d.exceptAll(df_nba_bnl_1d).head(1)) == 0)
display(len(df_nba_bnl_2d.exceptAll(df_nba_reference_2d).head(1)) == 0)
display(len(df_nba_reference_2d.exceptAll(df_nba_bnl_2d).head(1)) == 0)
display(len(df_nba_bnl_6d.exceptAll(df_nba_reference_6d).head(1)) == 0)
display(len(df_nba_reference_6d.exceptAll(df_nba_bnl_6d).head(1)) == 0)

display("distinct:")

display(len(df_airbnb_bnl_1d_distinct.select(airbnb_dims[:1]).exceptAll(df_airbnb_reference_1d_distinct.select(airbnb_dims[:1])).head(1)) == 0)
display(len(df_airbnb_reference_1d_distinct.select(airbnb_dims[:1]).exceptAll(df_airbnb_bnl_1d_distinct.select(airbnb_dims[:1])).head(1)) == 0)
display(len(df_airbnb_bnl_2d_distinct.select(airbnb_dims[:2]).exceptAll(df_airbnb_reference_2d_distinct.select(airbnb_dims[:2])).head(1)) == 0)
display(len(df_airbnb_reference_2d_distinct.select(airbnb_dims[:2]).exceptAll(df_airbnb_bnl_2d_distinct.select(airbnb_dims[:2])).head(1)) == 0)
display(len(df_airbnb_bnl_6d_distinct.select(airbnb_dims[:6]).exceptAll(df_airbnb_reference_6d_distinct.select(airbnb_dims[:6])).head(1)) == 0)
display(len(df_airbnb_reference_6d_distinct.select(airbnb_dims[:6]).exceptAll(df_airbnb_bnl_6d_distinct.select(airbnb_dims[:6])).head(1)) == 0)

display(len(df_fueleconomy_bnl_1d_distinct.select(fueleconomy_dims[:1]).exceptAll(df_fueleconomy_reference_1d_distinct.select(fueleconomy_dims[:1])).head(1)) == 0)
display(len(df_fueleconomy_reference_1d_distinct.select(fueleconomy_dims[:1]).exceptAll(df_fueleconomy_bnl_1d_distinct.select(fueleconomy_dims[:1])).head(1)) == 0)
display(len(df_fueleconomy_bnl_2d_distinct.select(fueleconomy_dims[:2]).exceptAll(df_fueleconomy_reference_2d_distinct.select(fueleconomy_dims[:2])).head(1)) == 0)
display(len(df_fueleconomy_reference_2d_distinct.select(fueleconomy_dims[:2]).exceptAll(df_fueleconomy_bnl_2d_distinct.select(fueleconomy_dims[:2])).head(1)) == 0)
display(len(df_fueleconomy_bnl_6d_distinct.select(fueleconomy_dims[:6]).exceptAll(df_fueleconomy_reference_6d_distinct.select(fueleconomy_dims[:6])).head(1)) == 0)
display(len(df_fueleconomy_reference_6d_distinct.select(fueleconomy_dims[:6]).exceptAll(df_fueleconomy_bnl_6d_distinct.select(fueleconomy_dims[:6])).head(1)) == 0)

display(len(df_coil2000_bnl_1d_distinct.select(coil2000_dims[:1]).exceptAll(df_coil2000_reference_1d_distinct.select(coil2000_dims[:1])).head(1)) == 0)
display(len(df_coil2000_reference_1d_distinct.select(coil2000_dims[:1]).exceptAll(df_coil2000_bnl_1d_distinct.select(coil2000_dims[:1])).head(1)) == 0)
display(len(df_coil2000_bnl_2d_distinct.select(coil2000_dims[:2]).exceptAll(df_coil2000_reference_2d_distinct.select(coil2000_dims[:2])).head(1)) == 0)
display(len(df_coil2000_reference_2d_distinct.select(coil2000_dims[:2]).exceptAll(df_coil2000_bnl_2d_distinct.select(coil2000_dims[:2])).head(1)) == 0)
display(len(df_coil2000_bnl_6d_distinct.select(coil2000_dims[:6]).exceptAll(df_coil2000_reference_6d_distinct.select(coil2000_dims[:6])).head(1)) == 0)
display(len(df_coil2000_reference_6d_distinct.select(coil2000_dims[:6]).exceptAll(df_coil2000_bnl_6d_distinct.select(coil2000_dims[:6])).head(1)) == 0)

display(len(df_nba_bnl_1d_distinct.select(nba_dims[:1]).exceptAll(df_nba_reference_1d_distinct.select(nba_dims[:1])).head(1)) == 0)
display(len(df_nba_reference_1d_distinct.select(nba_dims[:1]).exceptAll(df_nba_bnl_1d_distinct.select(nba_dims[:1])).head(1)) == 0)
display(len(df_nba_bnl_2d_distinct.select(nba_dims[:2]).exceptAll(df_nba_reference_2d_distinct.select(nba_dims[:2])).head(1)) == 0)

display(len(df_nba_reference_2d_distinct.select(nba_dims[:2]).exceptAll(df_nba_bnl_2d_distinct.select(nba_dims[:2])).head(1)) == 0)

display(len(df_nba_bnl_6d_distinct.select(nba_dims[:6]).exceptAll(df_nba_reference_6d_distinct.select(nba_dims[:6])).head(1)) == 0)
display(len(df_nba_reference_6d_distinct.select(nba_dims[:6]).exceptAll(df_nba_bnl_6d_distinct.select(nba_dims[:6])).head(1)) == 0)

## Skyline Solutions Distributed Skyline (Complete)

This section contains the results of using the distributed skyline algorithm.

### Non-Distinct Queries

In this subsection we calculate the results for each non-distinct skyline query. For distinct skyline queries see the next subsection.

In [ ]:
df_airbnb_dist_1d = sqlContext.sql("""
SELECT * FROM tests.airbnb_test SKYLINE OF COMPLETE
  price MIN
ORDER BY id
""")
df_airbnb_dist_1d.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_airbnb_dist_1d")

df_airbnb_dist_2d = sqlContext.sql("""
SELECT * FROM tests.airbnb_test SKYLINE OF COMPLETE
  price MIN,
  accommodates MAX
ORDER BY id
""")
df_airbnb_dist_2d.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_airbnb_dist_2d")

df_airbnb_dist_6d = sqlContext.sql("""
SELECT * FROM tests.airbnb_test SKYLINE OF COMPLETE
  price MIN,
  accommodates MAX,
  bedrooms MAX,
  beds MAX,
  number_of_reviews MAX,
  review_scores_rating MAX
ORDER BY id
""")
df_airbnb_dist_6d.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_airbnb_dist_6d")

In [ ]:
df_fueleconomy_dist_1d = sqlContext.sql("""
SELECT * FROM tests.fueleconomy_test SKYLINE OF COMPLETE
  fuelCost08 MIN
ORDER BY make, model
""")
df_fueleconomy_dist_1d.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_fueleconomy_dist_1d")

df_fueleconomy_dist_2d = sqlContext.sql("""
SELECT * FROM tests.fueleconomy_test SKYLINE OF COMPLETE
  fuelCost08 MIN,
  barrels08 MIN
ORDER BY make, model
""")
df_fueleconomy_dist_2d.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_fueleconomy_dist_2d")

df_fueleconomy_dist_6d = sqlContext.sql("""
SELECT * FROM tests.fueleconomy_test SKYLINE OF COMPLETE
  fuelCost08 MIN,
  barrels08 MIN,
  city08 MAX,
  highway08 MAX,
  comb08 MAX,
  combinedCD MIN
ORDER BY make, model
""")
df_fueleconomy_dist_6d.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_fueleconomy_dist_6d")

In [ ]:
df_coil2000_dist_1d = sqlContext.sql("""
SELECT * FROM tests.coil2000_test SKYLINE OF COMPLETE
  MOSHOOFD MAX
ORDER BY id
""")
df_coil2000_dist_1d.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_coil2000_dist_1d")

df_coil2000_dist_2d = sqlContext.sql("""
SELECT * FROM tests.coil2000_test SKYLINE OF COMPLETE
  MOSHOOFD MAX,
  MGODRK MIN
ORDER BY id
""")
df_coil2000_dist_2d.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_coil2000_dist_2d")

df_coil2000_dist_6d = sqlContext.sql("""
SELECT * FROM tests.coil2000_test SKYLINE OF COMPLETE
  MOSHOOFD MAX,
  MGODRK MIN,
  MGODPR MIN,
  MGODGE MAX,
  MRELGE MAX,
  MRELOV MAX
ORDER BY id
""")
df_coil2000_dist_6d.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_coil2000_dist_6d")

In [ ]:
df_nba_dist_1d = sqlContext.sql("""
SELECT * FROM tests.nba_test SKYLINE OF COMPLETE
  W MAX
ORDER BY PLAYER_ID
""")
df_nba_dist_1d.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_nba_dist_1d")

df_nba_dist_2d = sqlContext.sql("""
SELECT * FROM tests.nba_test SKYLINE OF COMPLETE
  W MAX,
  L MIN
ORDER BY PLAYER_ID
""")
df_nba_dist_2d.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_nba_dist_2d")

df_nba_dist_6d = sqlContext.sql("""
SELECT * FROM tests.nba_test SKYLINE OF COMPLETE
  W MAX,
  L MIN,
  W_PCT MAX,
  FGM MAX,
  FGA MAX,
  FG_PCT MAX
ORDER BY PLAYER_ID
""")
df_nba_dist_6d.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_nba_dist_6d")

### Distinct Queries

In this subsection we calculate the results for each distinct skyline query. For non-distinct skyline queries see the previous subsection.

In [ ]:
df_airbnb_dist_1d_distinct = sqlContext.sql("""
SELECT * FROM tests.airbnb_test SKYLINE OF DISTINCT COMPLETE
  price MIN
ORDER BY id
""")
df_airbnb_dist_1d_distinct.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_airbnb_dist_1d_distinct")

df_airbnb_dist_2d_distinct = sqlContext.sql("""
SELECT * FROM tests.airbnb_test SKYLINE OF DISTINCT COMPLETE
  price MIN,
  accommodates MAX
ORDER BY id
""")
df_airbnb_dist_2d_distinct.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_airbnb_dist_2d_distinct")

df_airbnb_dist_6d_distinct = sqlContext.sql("""
SELECT * FROM tests.airbnb_test SKYLINE OF DISTINCT COMPLETE
  price MIN,
  accommodates MAX,
  bedrooms MAX,
  beds MAX,
  number_of_reviews MAX,
  review_scores_rating MAX
ORDER BY id
""")
df_airbnb_dist_6d_distinct.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_airbnb_dist_6d_distinct")

In [ ]:
df_fueleconomy_dist_1d_distinct = sqlContext.sql("""
SELECT * FROM tests.fueleconomy_test SKYLINE OF DISTINCT COMPLETE
  fuelCost08 MIN
ORDER BY make, model
""")
df_fueleconomy_dist_1d_distinct.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_fueleconomy_dist_1d_distinct")

df_fueleconomy_dist_2d_distinct = sqlContext.sql("""
SELECT * FROM tests.fueleconomy_test SKYLINE OF DISTINCT COMPLETE
  fuelCost08 MIN,
  barrels08 MIN
ORDER BY make, model
""")
df_fueleconomy_dist_2d_distinct.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_fueleconomy_dist_2d_distinct")

df_fueleconomy_dist_6d_distinct = sqlContext.sql("""
SELECT * FROM tests.fueleconomy_test SKYLINE OF DISTINCT COMPLETE
  fuelCost08 MIN,
  barrels08 MIN,
  city08 MAX,
  highway08 MAX,
  comb08 MAX,
  combinedCD MIN
ORDER BY make, model
""")
df_fueleconomy_dist_6d_distinct.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_fueleconomy_dist_6d_distinct")

In [ ]:
df_coil2000_dist_1d_distinct = sqlContext.sql("""
SELECT * FROM tests.coil2000_test SKYLINE OF DISTINCT COMPLETE
  MOSHOOFD MAX
ORDER BY id
""")
df_coil2000_dist_1d_distinct.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_coil2000_dist_1d_distinct")

df_coil2000_dist_2d_distinct = sqlContext.sql("""
SELECT * FROM tests.coil2000_test SKYLINE OF DISTINCT COMPLETE
  MOSHOOFD MAX,
  MGODRK MIN
ORDER BY id
""")
df_coil2000_dist_2d_distinct.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_coil2000_dist_2d_distinct")

df_coil2000_dist_6d_distinct = sqlContext.sql("""
SELECT * FROM tests.coil2000_test SKYLINE OF DISTINCT COMPLETE
  MOSHOOFD MAX,
  MGODRK MIN,
  MGODPR MIN,
  MGODGE MAX,
  MRELGE MAX,
  MRELOV MAX
ORDER BY id
""")
df_coil2000_dist_6d_distinct.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_coil2000_dist_6d_distinct")

In [ ]:
df_nba_dist_1d_distinct = sqlContext.sql("""
SELECT * FROM tests.nba_test SKYLINE OF DISTINCT COMPLETE
  W MAX
ORDER BY PLAYER_ID
""")
df_nba_dist_1d_distinct.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_nba_dist_1d_distinct")

df_nba_dist_2d_distinct = sqlContext.sql("""
SELECT * FROM tests.nba_test SKYLINE OF DISTINCT COMPLETE
  W MAX,
  L MIN
ORDER BY PLAYER_ID
""")
df_nba_dist_2d_distinct.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_nba_dist_2d_distinct")

df_nba_dist_6d_distinct = sqlContext.sql("""
SELECT * FROM tests.nba_test SKYLINE OF DISTINCT COMPLETE
  W MAX,
  L MIN,
  W_PCT MAX,
  FGM MAX,
  FGA MAX,
  FG_PCT MAX
ORDER BY PLAYER_ID
""")
df_nba_dist_6d_distinct.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_nba_dist_6d_distinct")

### Result Area

In [ ]:
display("non-distinct:")

display(len(df_airbnb_dist_1d.exceptAll(df_airbnb_reference_1d).head(1)) == 0)
display(len(df_airbnb_reference_1d.exceptAll(df_airbnb_dist_1d).head(1)) == 0)
display(len(df_airbnb_dist_2d.exceptAll(df_airbnb_reference_2d).head(1)) == 0)
display(len(df_airbnb_reference_2d.exceptAll(df_airbnb_dist_2d).head(1)) == 0)
display(len(df_airbnb_dist_6d.exceptAll(df_airbnb_reference_6d).head(1)) == 0)
display(len(df_airbnb_reference_6d.exceptAll(df_airbnb_dist_6d).head(1)) == 0)

display(len(df_fueleconomy_dist_1d.exceptAll(df_fueleconomy_reference_1d).head(1)) == 0)
display(len(df_fueleconomy_reference_1d.exceptAll(df_fueleconomy_dist_1d).head(1)) == 0)
display(len(df_fueleconomy_dist_2d.exceptAll(df_fueleconomy_reference_2d).head(1)) == 0)
display(len(df_fueleconomy_reference_2d.exceptAll(df_fueleconomy_dist_2d).head(1)) == 0)
display(len(df_fueleconomy_dist_6d.exceptAll(df_fueleconomy_reference_6d).head(1)) == 0)
display(len(df_fueleconomy_reference_6d.exceptAll(df_fueleconomy_dist_6d).head(1)) == 0)

display(len(df_coil2000_dist_1d.exceptAll(df_coil2000_reference_1d).head(1)) == 0)
display(len(df_coil2000_reference_1d.exceptAll(df_coil2000_dist_1d).head(1)) == 0)
display(len(df_coil2000_dist_2d.exceptAll(df_coil2000_reference_2d).head(1)) == 0)
display(len(df_coil2000_reference_2d.exceptAll(df_coil2000_dist_2d).head(1)) == 0)
display(len(df_coil2000_dist_6d.exceptAll(df_coil2000_reference_6d).head(1)) == 0)
display(len(df_coil2000_reference_6d.exceptAll(df_coil2000_dist_6d).head(1)) == 0)

display(len(df_nba_dist_1d.exceptAll(df_nba_reference_1d).head(1)) == 0)
display(len(df_nba_reference_1d.exceptAll(df_nba_dist_1d).head(1)) == 0)
display(len(df_nba_dist_2d.exceptAll(df_nba_reference_2d).head(1)) == 0)
display(len(df_nba_reference_2d.exceptAll(df_nba_dist_2d).head(1)) == 0)
display(len(df_nba_dist_6d.exceptAll(df_nba_reference_6d).head(1)) == 0)
display(len(df_nba_reference_6d.exceptAll(df_nba_dist_6d).head(1)) == 0)

display("distinct:")

display(len(df_airbnb_dist_1d_distinct.select(airbnb_dims[:1]).exceptAll(df_airbnb_reference_1d_distinct.select(airbnb_dims[:1])).head(1)) == 0)
display(len(df_airbnb_reference_1d_distinct.select(airbnb_dims[:1]).exceptAll(df_airbnb_dist_1d_distinct.select(airbnb_dims[:1])).head(1)) == 0)
display(len(df_airbnb_dist_2d_distinct.select(airbnb_dims[:2]).exceptAll(df_airbnb_reference_2d_distinct.select(airbnb_dims[:2])).head(1)) == 0)
display(len(df_airbnb_reference_2d_distinct.select(airbnb_dims[:2]).exceptAll(df_airbnb_dist_2d_distinct.select(airbnb_dims[:2])).head(1)) == 0)
display(len(df_airbnb_dist_6d_distinct.select(airbnb_dims[:6]).exceptAll(df_airbnb_reference_6d_distinct.select(airbnb_dims[:6])).head(1)) == 0)
display(len(df_airbnb_reference_6d_distinct.select(airbnb_dims[:6]).exceptAll(df_airbnb_dist_6d_distinct.select(airbnb_dims[:6])).head(1)) == 0)

display(len(df_fueleconomy_dist_1d_distinct.select(fueleconomy_dims[:1]).exceptAll(df_fueleconomy_reference_1d_distinct.select(fueleconomy_dims[:1])).head(1)) == 0)
display(len(df_fueleconomy_reference_1d_distinct.select(fueleconomy_dims[:1]).exceptAll(df_fueleconomy_dist_1d_distinct.select(fueleconomy_dims[:1])).head(1)) == 0)
display(len(df_fueleconomy_dist_2d_distinct.select(fueleconomy_dims[:2]).exceptAll(df_fueleconomy_reference_2d_distinct.select(fueleconomy_dims[:2])).head(1)) == 0)
display(len(df_fueleconomy_reference_2d_distinct.select(fueleconomy_dims[:2]).exceptAll(df_fueleconomy_dist_2d_distinct.select(fueleconomy_dims[:2])).head(1)) == 0)
display(len(df_fueleconomy_dist_6d_distinct.select(fueleconomy_dims[:6]).exceptAll(df_fueleconomy_reference_6d_distinct.select(fueleconomy_dims[:6])).head(1)) == 0)
display(len(df_fueleconomy_reference_6d_distinct.select(fueleconomy_dims[:6]).exceptAll(df_fueleconomy_dist_6d_distinct.select(fueleconomy_dims[:6])).head(1)) == 0)

display(len(df_coil2000_dist_1d_distinct.select(coil2000_dims[:1]).exceptAll(df_coil2000_reference_1d_distinct.select(coil2000_dims[:1])).head(1)) == 0)
display(len(df_coil2000_reference_1d_distinct.select(coil2000_dims[:1]).exceptAll(df_coil2000_dist_1d_distinct.select(coil2000_dims[:1])).head(1)) == 0)
display(len(df_coil2000_dist_2d_distinct.select(coil2000_dims[:2]).exceptAll(df_coil2000_reference_2d_distinct.select(coil2000_dims[:2])).head(1)) == 0)
display(len(df_coil2000_reference_2d_distinct.select(coil2000_dims[:2]).exceptAll(df_coil2000_dist_2d_distinct.select(coil2000_dims[:2])).head(1)) == 0)
display(len(df_coil2000_dist_6d_distinct.select(coil2000_dims[:6]).exceptAll(df_coil2000_reference_6d_distinct.select(coil2000_dims[:6])).head(1)) == 0)
display(len(df_coil2000_reference_6d_distinct.select(coil2000_dims[:6]).exceptAll(df_coil2000_dist_6d_distinct.select(coil2000_dims[:6])).head(1)) == 0)

display(len(df_nba_dist_1d_distinct.select(nba_dims[:1]).exceptAll(df_nba_reference_1d_distinct.select(nba_dims[:1])).head(1)) == 0)
display(len(df_nba_reference_1d_distinct.select(nba_dims[:1]).exceptAll(df_nba_dist_1d_distinct.select(nba_dims[:1])).head(1)) == 0)
display(len(df_nba_dist_2d_distinct.select(nba_dims[:2]).exceptAll(df_nba_reference_2d_distinct.select(nba_dims[:2])).head(1)) == 0)
display(len(df_nba_reference_2d_distinct.select(nba_dims[:2]).exceptAll(df_nba_dist_2d_distinct.select(nba_dims[:2])).head(1)) == 0)
display(len(df_nba_dist_6d_distinct.select(nba_dims[:6]).exceptAll(df_nba_reference_6d_distinct.select(nba_dims[:6])).head(1)) == 0)
display(len(df_nba_reference_6d_distinct.select(nba_dims[:6]).exceptAll(df_nba_dist_6d_distinct.select(nba_dims[:6])).head(1)) == 0)

## Skyline Solutions Distributed Skyline (Incomplete)

This section contains the results of using the **incomplete** distributed skyline algorithm.

### Tests performed on the complete dataset

This subsection contains the tests performed on the complete dataset as opposed to incomplete datasets which can be found in the next subsection.

#### Non-Distinct Queries

In this subsection we calculate the results for each non-distinct skyline query. For distinct skyline queries see the next subsection.

In [ ]:
df_airbnb_dist_inc_1d = sqlContext.sql("""
SELECT * FROM tests.airbnb_test SKYLINE OF
  price MIN
ORDER BY id
""")
df_airbnb_dist_inc_1d.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_airbnb_dist_inc_1d")

df_airbnb_dist_inc_2d = sqlContext.sql("""
SELECT * FROM tests.airbnb_test SKYLINE OF
  price MIN,
  accommodates MAX
ORDER BY id
""")
df_airbnb_dist_inc_2d.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_airbnb_dist_inc_2d")

df_airbnb_dist_inc_6d = sqlContext.sql("""
SELECT * FROM tests.airbnb_test SKYLINE OF
  price MIN,
  accommodates MAX,
  bedrooms MAX,
  beds MAX,
  number_of_reviews MAX,
  review_scores_rating MAX
ORDER BY id
""")
df_airbnb_dist_inc_6d.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_airbnb_dist_inc_6d")

In [ ]:
df_fueleconomy_dist_inc_1d = sqlContext.sql("""
SELECT * FROM tests.fueleconomy_test SKYLINE OF
  fuelCost08 MIN
ORDER BY make, model
""")
df_fueleconomy_dist_inc_1d.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_fueleconomy_dist_inc_1d")

df_fueleconomy_dist_inc_2d = sqlContext.sql("""
SELECT * FROM tests.fueleconomy_test SKYLINE OF
  fuelCost08 MIN,
  barrels08 MIN
ORDER BY make, model
""")
df_fueleconomy_dist_inc_2d.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_fueleconomy_dist_inc_2d")

df_fueleconomy_dist_inc_6d = sqlContext.sql("""
SELECT * FROM tests.fueleconomy_test SKYLINE OF
  fuelCost08 MIN,
  barrels08 MIN,
  city08 MAX,
  highway08 MAX,
  comb08 MAX,
  combinedCD MIN
ORDER BY make, model
""")
df_fueleconomy_dist_inc_6d.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_fueleconomy_dist_inc_6d")

In [ ]:
df_coil2000_dist_inc_1d = sqlContext.sql("""
SELECT * FROM tests.coil2000_test SKYLINE OF
  MOSHOOFD MAX
ORDER BY id
""")
df_coil2000_dist_inc_1d.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_coil2000_dist_inc_1d")

df_coil2000_dist_inc_2d = sqlContext.sql("""
SELECT * FROM tests.coil2000_test SKYLINE OF
  MOSHOOFD MAX,
  MGODRK MIN
ORDER BY id
""")
df_coil2000_dist_inc_2d.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_coil2000_dist_inc_2d")

df_coil2000_dist_inc_6d = sqlContext.sql("""
SELECT * FROM tests.coil2000_test SKYLINE OF
  MOSHOOFD MAX,
  MGODRK MIN,
  MGODPR MIN,
  MGODGE MAX,
  MRELGE MAX,
  MRELOV MAX
ORDER BY id
""")
df_coil2000_dist_inc_6d.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_coil2000_dist_inc_6d")

In [ ]:
df_nba_dist_inc_1d = sqlContext.sql("""
SELECT * FROM tests.nba_test SKYLINE OF
  W MAX
ORDER BY PLAYER_ID
""")
df_nba_dist_inc_1d.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_nba_dist_inc_1d")

df_nba_dist_inc_2d = sqlContext.sql("""
SELECT * FROM tests.nba_test SKYLINE OF
  W MAX,
  L MIN
ORDER BY PLAYER_ID
""")
df_nba_dist_inc_2d.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_nba_dist_inc_2d")

df_nba_dist_inc_6d = sqlContext.sql("""
SELECT * FROM tests.nba_test SKYLINE OF
  W MAX,
  L MIN,
  W_PCT MAX,
  FGM MAX,
  FGA MAX,
  FG_PCT MAX
ORDER BY PLAYER_ID
""")
df_nba_dist_inc_6d.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_nba_dist_inc_6d")

#### Distinct Queries

In this subsection we calculate the results for each distinct skyline query. For non-distinct skyline queries see the previous subsection.

In [ ]:
df_airbnb_dist_inc_1d_distinct = sqlContext.sql("""
SELECT * FROM tests.airbnb_test SKYLINE OF DISTINCT
  price MIN
ORDER BY id
""")
df_airbnb_dist_inc_1d_distinct.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_airbnb_dist_inc_1d_distinct")

df_airbnb_dist_inc_2d_distinct = sqlContext.sql("""
SELECT * FROM tests.airbnb_test SKYLINE OF DISTINCT
  price MIN,
  accommodates MAX
ORDER BY id
""")
df_airbnb_dist_inc_2d_distinct.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_airbnb_dist_inc_2d_distinct")

df_airbnb_dist_inc_6d_distinct = sqlContext.sql("""
SELECT * FROM tests.airbnb_test SKYLINE OF DISTINCT
  price MIN,
  accommodates MAX,
  bedrooms MAX,
  beds MAX,
  number_of_reviews MAX,
  review_scores_rating MAX
ORDER BY id
""")
df_airbnb_dist_inc_6d_distinct.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_airbnb_dist_inc_6d_distinct")

In [ ]:
df_fueleconomy_dist_inc_1d_distinct = sqlContext.sql("""
SELECT * FROM tests.fueleconomy_test SKYLINE OF DISTINCT
  fuelCost08 MIN
ORDER BY make, model
""")
df_fueleconomy_dist_inc_1d_distinct.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_fueleconomy_dist_inc_1d_distinct")

df_fueleconomy_dist_inc_2d_distinct = sqlContext.sql("""
SELECT * FROM tests.fueleconomy_test SKYLINE OF DISTINCT
  fuelCost08 MIN,
  barrels08 MIN
ORDER BY make, model
""")
df_fueleconomy_dist_inc_2d_distinct.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_fueleconomy_dist_inc_2d_distinct")

df_fueleconomy_dist_inc_6d_distinct = sqlContext.sql("""
SELECT * FROM tests.fueleconomy_test SKYLINE OF DISTINCT
  fuelCost08 MIN,
  barrels08 MIN,
  city08 MAX,
  highway08 MAX,
  comb08 MAX,
  combinedCD MIN
ORDER BY make, model
""")
df_fueleconomy_dist_inc_6d_distinct.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_fueleconomy_dist_inc_6d_distinct")

In [ ]:
df_coil2000_dist_inc_1d_distinct = sqlContext.sql("""
SELECT * FROM tests.coil2000_test SKYLINE OF DISTINCT
  MOSHOOFD MAX
ORDER BY id
""")
df_coil2000_dist_inc_1d_distinct.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_coil2000_dist_inc_1d_distinct")

df_coil2000_dist_inc_2d_distinct = sqlContext.sql("""
SELECT * FROM tests.coil2000_test SKYLINE OF DISTINCT
  MOSHOOFD MAX,
  MGODRK MIN
ORDER BY id
""")
df_coil2000_dist_inc_2d_distinct.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_coil2000_dist_inc_2d_distinct")

df_coil2000_dist_inc_6d_distinct = sqlContext.sql("""
SELECT * FROM tests.coil2000_test SKYLINE OF DISTINCT
  MOSHOOFD MAX,
  MGODRK MIN,
  MGODPR MIN,
  MGODGE MAX,
  MRELGE MAX,
  MRELOV MAX
ORDER BY id
""")
df_coil2000_dist_inc_6d_distinct.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_coil2000_dist_inc_6d_distinct")

In [ ]:
df_nba_dist_inc_1d_distinct = sqlContext.sql("""
SELECT * FROM tests.nba_test SKYLINE OF DISTINCT
  W MAX
ORDER BY PLAYER_ID
""")
df_nba_dist_inc_1d_distinct.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_nba_dist_inc_1d_distinct")

df_nba_dist_inc_2d_distinct = sqlContext.sql("""
SELECT * FROM tests.nba_test SKYLINE OF DISTINCT
  W MAX,
  L MIN
ORDER BY PLAYER_ID
""")
df_nba_dist_inc_2d_distinct.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_nba_dist_inc_2d_distinct")

df_nba_dist_inc_6d_distinct = sqlContext.sql("""
SELECT * FROM tests.nba_test SKYLINE OF DISTINCT
  W MAX,
  L MIN,
  W_PCT MAX,
  FGM MAX,
  FGA MAX,
  FG_PCT MAX
ORDER BY PLAYER_ID
""")
df_nba_dist_inc_6d_distinct.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_nba_dist_inc_6d_distinct")

#### Result Area

In [ ]:
display("non-distinct:")

display(len(df_airbnb_dist_inc_1d.exceptAll(df_airbnb_reference_1d).head(1)) == 0)
display(len(df_airbnb_reference_1d.exceptAll(df_airbnb_dist_inc_1d).head(1)) == 0)
display(len(df_airbnb_dist_inc_2d.exceptAll(df_airbnb_reference_2d).head(1)) == 0)
display(len(df_airbnb_reference_2d.exceptAll(df_airbnb_dist_inc_2d).head(1)) == 0)
display(len(df_airbnb_dist_inc_6d.exceptAll(df_airbnb_reference_6d).head(1)) == 0)
display(len(df_airbnb_reference_6d.exceptAll(df_airbnb_dist_inc_6d).head(1)) == 0)

display(len(df_fueleconomy_dist_inc_1d.exceptAll(df_fueleconomy_reference_1d).head(1)) == 0)
display(len(df_fueleconomy_reference_1d.exceptAll(df_fueleconomy_dist_inc_1d).head(1)) == 0)
display(len(df_fueleconomy_dist_inc_2d.exceptAll(df_fueleconomy_reference_2d).head(1)) == 0)
display(len(df_fueleconomy_reference_2d.exceptAll(df_fueleconomy_dist_inc_2d).head(1)) == 0)
display(len(df_fueleconomy_dist_inc_6d.exceptAll(df_fueleconomy_reference_6d).head(1)) == 0)
display(len(df_fueleconomy_reference_6d.exceptAll(df_fueleconomy_dist_inc_6d).head(1)) == 0)

display(len(df_coil2000_dist_inc_1d.exceptAll(df_coil2000_reference_1d).head(1)) == 0)
display(len(df_coil2000_reference_1d.exceptAll(df_coil2000_dist_inc_1d).head(1)) == 0)
display(len(df_coil2000_dist_inc_2d.exceptAll(df_coil2000_reference_2d).head(1)) == 0)
display(len(df_coil2000_reference_2d.exceptAll(df_coil2000_dist_inc_2d).head(1)) == 0)
display(len(df_coil2000_dist_inc_6d.exceptAll(df_coil2000_reference_6d).head(1)) == 0)
display(len(df_coil2000_reference_6d.exceptAll(df_coil2000_dist_inc_6d).head(1)) == 0)

display(len(df_nba_dist_inc_1d.exceptAll(df_nba_reference_1d).head(1)) == 0)
display(len(df_nba_reference_1d.exceptAll(df_nba_dist_inc_1d).head(1)) == 0)
display(len(df_nba_dist_inc_2d.exceptAll(df_nba_reference_2d).head(1)) == 0)
display(len(df_nba_reference_2d.exceptAll(df_nba_dist_inc_2d).head(1)) == 0)
display(len(df_nba_dist_inc_6d.exceptAll(df_nba_reference_6d).head(1)) == 0)
display(len(df_nba_reference_6d.exceptAll(df_nba_dist_inc_6d).head(1)) == 0)

display("distinct:")

display(len(df_airbnb_dist_inc_1d_distinct.select(airbnb_dims[:1]).exceptAll(df_airbnb_reference_1d_distinct.select(airbnb_dims[:1])).head(1)) == 0)
display(len(df_airbnb_reference_1d_distinct.select(airbnb_dims[:1]).exceptAll(df_airbnb_dist_inc_1d_distinct.select(airbnb_dims[:1])).head(1)) == 0)
display(len(df_airbnb_dist_inc_2d_distinct.select(airbnb_dims[:2]).exceptAll(df_airbnb_reference_2d_distinct.select(airbnb_dims[:2])).head(1)) == 0)
display(len(df_airbnb_reference_2d_distinct.select(airbnb_dims[:2]).exceptAll(df_airbnb_dist_inc_2d_distinct.select(airbnb_dims[:2])).head(1)) == 0)
display(len(df_airbnb_dist_inc_6d_distinct.select(airbnb_dims[:6]).exceptAll(df_airbnb_reference_6d_distinct.select(airbnb_dims[:6])).head(1)) == 0)
display(len(df_airbnb_reference_6d_distinct.select(airbnb_dims[:6]).exceptAll(df_airbnb_dist_inc_6d_distinct.select(airbnb_dims[:6])).head(1)) == 0)

display(len(df_fueleconomy_dist_inc_1d_distinct.select(fueleconomy_dims[:1]).exceptAll(df_fueleconomy_reference_1d_distinct.select(fueleconomy_dims[:1])).head(1)) == 0)
display(len(df_fueleconomy_reference_1d_distinct.select(fueleconomy_dims[:1]).exceptAll(df_fueleconomy_dist_inc_1d_distinct.select(fueleconomy_dims[:1])).head(1)) == 0)
display(len(df_fueleconomy_dist_inc_2d_distinct.select(fueleconomy_dims[:2]).exceptAll(df_fueleconomy_reference_2d_distinct.select(fueleconomy_dims[:2])).head(1)) == 0)
display(len(df_fueleconomy_reference_2d_distinct.select(fueleconomy_dims[:2]).exceptAll(df_fueleconomy_dist_inc_2d_distinct.select(fueleconomy_dims[:2])).head(1)) == 0)
display(len(df_fueleconomy_dist_inc_6d_distinct.select(fueleconomy_dims[:6]).exceptAll(df_fueleconomy_reference_6d_distinct.select(fueleconomy_dims[:6])).head(1)) == 0)
display(len(df_fueleconomy_reference_6d_distinct.select(fueleconomy_dims[:6]).exceptAll(df_fueleconomy_dist_inc_6d_distinct.select(fueleconomy_dims[:6])).head(1)) == 0)

display(len(df_coil2000_dist_inc_1d_distinct.select(coil2000_dims[:1]).exceptAll(df_coil2000_reference_1d_distinct.select(coil2000_dims[:1])).head(1)) == 0)
display(len(df_coil2000_reference_1d_distinct.select(coil2000_dims[:1]).exceptAll(df_coil2000_dist_inc_1d_distinct.select(coil2000_dims[:1])).head(1)) == 0)
display(len(df_coil2000_dist_inc_2d_distinct.select(coil2000_dims[:2]).exceptAll(df_coil2000_reference_2d_distinct.select(coil2000_dims[:2])).head(1)) == 0)
display(len(df_coil2000_reference_2d_distinct.select(coil2000_dims[:2]).exceptAll(df_coil2000_dist_inc_2d_distinct.select(coil2000_dims[:2])).head(1)) == 0)
display(len(df_coil2000_dist_inc_6d_distinct.select(coil2000_dims[:6]).exceptAll(df_coil2000_reference_6d_distinct.select(coil2000_dims[:6])).head(1)) == 0)
display(len(df_coil2000_reference_6d_distinct.select(coil2000_dims[:6]).exceptAll(df_coil2000_dist_inc_6d_distinct.select(coil2000_dims[:6])).head(1)) == 0)

display(len(df_nba_dist_inc_1d_distinct.select(nba_dims[:1]).exceptAll(df_nba_reference_1d_distinct.select(nba_dims[:1])).head(1)) == 0)
display(len(df_nba_reference_1d_distinct.select(nba_dims[:1]).exceptAll(df_nba_dist_inc_1d_distinct.select(nba_dims[:1])).head(1)) == 0)
display(len(df_nba_dist_inc_2d_distinct.select(nba_dims[:2]).exceptAll(df_nba_reference_2d_distinct.select(nba_dims[:2])).head(1)) == 0)
display(len(df_nba_reference_2d_distinct.select(nba_dims[:2]).exceptAll(df_nba_dist_inc_2d_distinct.select(nba_dims[:2])).head(1)) == 0)
display(len(df_nba_dist_inc_6d_distinct.select(nba_dims[:6]).exceptAll(df_nba_reference_6d_distinct.select(nba_dims[:6])).head(1)) == 0)
display(len(df_nba_reference_6d_distinct.select(nba_dims[:6]).exceptAll(df_nba_dist_inc_6d_distinct.select(nba_dims[:6])).head(1)) == 0)

### Tests performed on the **incomplete** dataset

This subsection contains the tests performed on the **incomplete** dataset as opposed to complete datasets which can be found in the previous subsection.

#### Non-Distinct Queries

In this subsection we calculate the results for each non-distinct skyline query. For distinct skyline queries see the next subsection.

In [ ]:
df_airbnb_incomplete_dist_inc_1d = sqlContext.sql("""
SELECT * FROM tests.airbnb_test_incomplete SKYLINE OF
  price MIN
ORDER BY id
""")
df_airbnb_incomplete_dist_inc_1d.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_airbnb_incomplete_dist_inc_1d")

df_airbnb_incomplete_dist_inc_2d = sqlContext.sql("""
SELECT * FROM tests.airbnb_test_incomplete SKYLINE OF
  price MIN,
  accommodates MAX
ORDER BY id
""")
df_airbnb_incomplete_dist_inc_2d.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_airbnb_incomplete_dist_inc_2d")

df_airbnb_incomplete_dist_inc_6d = sqlContext.sql("""
SELECT * FROM tests.airbnb_test_incomplete SKYLINE OF
  price MIN,
  accommodates MAX,
  bedrooms MAX,
  beds MAX,
  number_of_reviews MAX,
  review_scores_rating MAX
ORDER BY id
""")
df_airbnb_incomplete_dist_inc_6d.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_airbnb_incomplete_dist_inc_6d")

In [ ]:
df_fueleconomy_incomplete_dist_inc_1d = sqlContext.sql("""
SELECT * FROM tests.fueleconomy_test_incomplete SKYLINE OF
  fuelCost08 MIN
ORDER BY make, model
""")
df_fueleconomy_incomplete_dist_inc_1d.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_fueleconomy_incomplete_dist_inc_1d")

df_fueleconomy_incomplete_dist_inc_2d = sqlContext.sql("""
SELECT * FROM tests.fueleconomy_test_incomplete SKYLINE OF
  fuelCost08 MIN,
  barrels08 MIN
ORDER BY make, model
""")
df_fueleconomy_incomplete_dist_inc_2d.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_fueleconomy_incomplete_dist_inc_2d")

df_fueleconomy_incomplete_dist_inc_6d = sqlContext.sql("""
SELECT * FROM tests.fueleconomy_test_incomplete SKYLINE OF
  fuelCost08 MIN,
  barrels08 MIN,
  city08 MAX,
  highway08 MAX,
  comb08 MAX,
  combinedCD MIN
ORDER BY make, model
""")
df_fueleconomy_incomplete_dist_inc_6d.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_fueleconomy_incomplete_dist_inc_6d")

In [ ]:
df_coil2000_incomplete_dist_inc_1d = sqlContext.sql("""
SELECT * FROM tests.coil2000_test_incomplete SKYLINE OF
  MOSHOOFD MAX
ORDER BY id
""")
df_coil2000_incomplete_dist_inc_1d.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_coil2000_incomplete_dist_inc_1d")

df_coil2000_incomplete_dist_inc_2d = sqlContext.sql("""
SELECT * FROM tests.coil2000_test_incomplete SKYLINE OF
  MOSHOOFD MAX,
  MGODRK MIN
ORDER BY id
""")
df_coil2000_incomplete_dist_inc_2d.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_coil2000_incomplete_dist_inc_2d")

df_coil2000_incomplete_dist_inc_6d = sqlContext.sql("""
SELECT * FROM tests.coil2000_test_incomplete SKYLINE OF
  MOSHOOFD MAX,
  MGODRK MIN,
  MGODPR MIN,
  MGODGE MAX,
  MRELGE MAX,
  MRELOV MAX
ORDER BY id
""")
df_coil2000_incomplete_dist_inc_6d.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_coil2000_incomplete_dist_inc_6d")

In [ ]:
df_nba_incomplete_dist_inc_1d = sqlContext.sql("""
SELECT * FROM tests.nba_test_incomplete SKYLINE OF
  W MAX
ORDER BY PLAYER_ID
""")
df_nba_incomplete_dist_inc_1d.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_nba_incomplete_dist_inc_1d")

df_nba_incomplete_dist_inc_2d = sqlContext.sql("""
SELECT * FROM tests.nba_test_incomplete SKYLINE OF
  W MAX,
  L MIN
ORDER BY PLAYER_ID
""")
df_nba_incomplete_dist_inc_2d.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_nba_incomplete_dist_inc_2d")

df_nba_incomplete_dist_inc_6d = sqlContext.sql("""
SELECT * FROM tests.nba_test_incomplete SKYLINE OF
  W MAX,
  L MIN,
  W_PCT MAX,
  FGM MAX,
  FGA MAX,
  FG_PCT MAX
ORDER BY PLAYER_ID
""")
df_nba_incomplete_dist_inc_6d.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_nba_incomplete_dist_inc_6d")

#### Distinct Queries

In this subsection we calculate the results for each distinct skyline query. For non-distinct skyline queries see the previous subsection.

In [ ]:
df_airbnb_incomplete_dist_inc_1d_distinct = sqlContext.sql("""
SELECT * FROM tests.airbnb_test_incomplete SKYLINE OF DISTINCT
  price MIN
ORDER BY id
""")
df_airbnb_incomplete_dist_inc_1d_distinct.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_airbnb_incomplete_dist_inc_1d_distinct")

df_airbnb_incomplete_dist_inc_2d_distinct = sqlContext.sql("""
SELECT * FROM tests.airbnb_test_incomplete SKYLINE OF DISTINCT
  price MIN,
  accommodates MAX
ORDER BY id
""")
df_airbnb_incomplete_dist_inc_2d_distinct.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_airbnb_incomplete_dist_inc_2d_distinct")

df_airbnb_incomplete_dist_inc_6d_distinct = sqlContext.sql("""
SELECT * FROM tests.airbnb_test_incomplete SKYLINE OF DISTINCT
  price MIN,
  accommodates MAX,
  bedrooms MAX,
  beds MAX,
  number_of_reviews MAX,
  review_scores_rating MAX
ORDER BY id
""")
df_airbnb_incomplete_dist_inc_6d_distinct.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_airbnb_incomplete_dist_inc_6d_distinct")

In [ ]:
df_fueleconomy_incomplete_dist_inc_1d_distinct = sqlContext.sql("""
SELECT * FROM tests.fueleconomy_test_incomplete SKYLINE OF DISTINCT
  fuelCost08 MIN
ORDER BY make, model
""")
df_fueleconomy_incomplete_dist_inc_1d_distinct.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_fueleconomy_incomplete_dist_inc_1d_distinct")

df_fueleconomy_incomplete_dist_inc_2d_distinct = sqlContext.sql("""
SELECT * FROM tests.fueleconomy_test_incomplete SKYLINE OF DISTINCT
  fuelCost08 MIN,
  barrels08 MIN
ORDER BY make, model
""")
df_fueleconomy_incomplete_dist_inc_2d_distinct.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_fueleconomy_incomplete_dist_inc_2d_distinct")

df_fueleconomy_incomplete_dist_inc_6d_distinct = sqlContext.sql("""
SELECT * FROM tests.fueleconomy_test_incomplete SKYLINE OF DISTINCT
  fuelCost08 MIN,
  barrels08 MIN,
  city08 MAX,
  highway08 MAX,
  comb08 MAX,
  combinedCD MIN
ORDER BY make, model
""")
df_fueleconomy_incomplete_dist_inc_6d_distinct.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_fueleconomy_incomplete_dist_inc_6d_distinct")

In [ ]:
df_coil2000_incomplete_dist_inc_1d_distinct = sqlContext.sql("""
SELECT * FROM tests.coil2000_test_incomplete SKYLINE OF DISTINCT
  MOSHOOFD MAX
ORDER BY id
""")
df_coil2000_incomplete_dist_inc_1d_distinct.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_coil2000_incomplete_dist_inc_1d_distinct")

df_coil2000_incomplete_dist_inc_2d_distinct = sqlContext.sql("""
SELECT * FROM tests.coil2000_test_incomplete SKYLINE OF DISTINCT
  MOSHOOFD MAX,
  MGODRK MIN
ORDER BY id
""")
df_coil2000_incomplete_dist_inc_2d_distinct.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_coil2000_incomplete_dist_inc_2d_distinct")

df_coil2000_incomplete_dist_inc_6d_distinct = sqlContext.sql("""
SELECT * FROM tests.coil2000_test_incomplete SKYLINE OF DISTINCT
  MOSHOOFD MAX,
  MGODRK MIN,
  MGODPR MIN,
  MGODGE MAX,
  MRELGE MAX,
  MRELOV MAX
ORDER BY id
""")
df_coil2000_incomplete_dist_inc_6d_distinct.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_coil2000_incomplete_dist_inc_6d_distinct")

In [ ]:
df_nba_incomplete_dist_inc_1d_distinct = sqlContext.sql("""
SELECT * FROM tests.nba_test_incomplete SKYLINE OF DISTINCT
  W MAX
ORDER BY PLAYER_ID
""")
df_nba_incomplete_dist_inc_1d_distinct.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_nba_incomplete_dist_inc_1d_distinct")

df_nba_incomplete_dist_inc_2d_distinct = sqlContext.sql("""
SELECT * FROM tests.nba_test_incomplete SKYLINE OF DISTINCT
  W MAX,
  L MIN
ORDER BY PLAYER_ID
""")
df_nba_incomplete_dist_inc_2d_distinct.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_nba_incomplete_dist_inc_2d_distinct")

df_nba_incomplete_dist_inc_6d_distinct = sqlContext.sql("""
SELECT * FROM tests.nba_test_incomplete SKYLINE OF DISTINCT
  W MAX,
  L MIN,
  W_PCT MAX,
  FGM MAX,
  FGA MAX,
  FG_PCT MAX
ORDER BY PLAYER_ID
""")
df_nba_incomplete_dist_inc_6d_distinct.repartition(1).write.option("header", "true").mode("overwrite").csv("./output/df_nba_incomplete_dist_inc_6d_distinct")

#### Result Area

In [ ]:
display("non-distinct:")

display(len(df_airbnb_incomplete_dist_inc_1d.exceptAll(df_airbnb_incomplete_reference_1d).head(1)) == 0)
display(len(df_airbnb_incomplete_reference_1d.exceptAll(df_airbnb_incomplete_dist_inc_1d).head(1)) == 0)
display(len(df_airbnb_incomplete_dist_inc_2d.exceptAll(df_airbnb_incomplete_reference_2d).head(1)) == 0)
display(len(df_airbnb_incomplete_reference_2d.exceptAll(df_airbnb_incomplete_dist_inc_2d).head(1)) == 0)
display(len(df_airbnb_incomplete_dist_inc_6d.exceptAll(df_airbnb_incomplete_reference_6d).head(1)) == 0)
display(len(df_airbnb_incomplete_reference_6d.exceptAll(df_airbnb_incomplete_dist_inc_6d).head(1)) == 0)

display(len(df_fueleconomy_incomplete_dist_inc_1d.exceptAll(df_fueleconomy_incomplete_reference_1d).head(1)) == 0)
display(len(df_fueleconomy_incomplete_reference_1d.exceptAll(df_fueleconomy_incomplete_dist_inc_1d).head(1)) == 0)
display(len(df_fueleconomy_incomplete_dist_inc_2d.exceptAll(df_fueleconomy_incomplete_reference_2d).head(1)) == 0)
display(len(df_fueleconomy_incomplete_reference_2d.exceptAll(df_fueleconomy_incomplete_dist_inc_2d).head(1)) == 0)
display(len(df_fueleconomy_incomplete_dist_inc_6d.exceptAll(df_fueleconomy_incomplete_reference_6d).head(1)) == 0)
display(len(df_fueleconomy_incomplete_reference_6d.exceptAll(df_fueleconomy_incomplete_dist_inc_6d).head(1)) == 0)

display(len(df_coil2000_incomplete_dist_inc_1d.exceptAll(df_coil2000_incomplete_reference_1d).head(1)) == 0)
display(len(df_coil2000_incomplete_reference_1d.exceptAll(df_coil2000_incomplete_dist_inc_1d).head(1)) == 0)
display(len(df_coil2000_incomplete_dist_inc_2d.exceptAll(df_coil2000_incomplete_reference_2d).head(1)) == 0)
display(len(df_coil2000_incomplete_reference_2d.exceptAll(df_coil2000_incomplete_dist_inc_2d).head(1)) == 0)
display(len(df_coil2000_incomplete_dist_inc_6d.exceptAll(df_coil2000_incomplete_reference_6d).head(1)) == 0)
display(len(df_coil2000_incomplete_reference_6d.exceptAll(df_coil2000_incomplete_dist_inc_6d).head(1)) == 0)

display(len(df_nba_incomplete_dist_inc_1d.exceptAll(df_nba_incomplete_reference_1d).head(1)) == 0)
display(len(df_nba_incomplete_reference_1d.exceptAll(df_nba_incomplete_dist_inc_1d).head(1)) == 0)
display(len(df_nba_incomplete_dist_inc_2d.exceptAll(df_nba_incomplete_reference_2d).head(1)) == 0)
display(len(df_nba_incomplete_reference_2d.exceptAll(df_nba_incomplete_dist_inc_2d).head(1)) == 0)
display(len(df_nba_incomplete_dist_inc_6d.exceptAll(df_nba_incomplete_reference_6d).head(1)) == 0)
display(len(df_nba_incomplete_reference_6d.exceptAll(df_nba_incomplete_dist_inc_6d).head(1)) == 0)

display("distinct:")

display(len(df_airbnb_incomplete_dist_inc_1d_distinct.select(airbnb_dims[:1]).exceptAll(df_airbnb_incomplete_reference_1d_distinct.select(airbnb_dims[:1])).head(1)) == 0)
display(len(df_airbnb_incomplete_reference_1d_distinct.select(airbnb_dims[:1]).exceptAll(df_airbnb_incomplete_dist_inc_1d_distinct.select(airbnb_dims[:1])).head(1)) == 0)
display(len(df_airbnb_incomplete_dist_inc_2d_distinct.select(airbnb_dims[:2]).exceptAll(df_airbnb_incomplete_reference_2d_distinct.select(airbnb_dims[:2])).head(1)) == 0)
display(len(df_airbnb_incomplete_reference_2d_distinct.select(airbnb_dims[:2]).exceptAll(df_airbnb_incomplete_dist_inc_2d_distinct.select(airbnb_dims[:2])).head(1)) == 0)
display(len(df_airbnb_incomplete_dist_inc_6d_distinct.select(airbnb_dims[:6]).exceptAll(df_airbnb_incomplete_reference_6d_distinct.select(airbnb_dims[:6])).head(1)) == 0)
display(len(df_airbnb_incomplete_reference_6d_distinct.select(airbnb_dims[:6]).exceptAll(df_airbnb_incomplete_dist_inc_6d_distinct.select(airbnb_dims[:6])).head(1)) == 0)

display(len(df_fueleconomy_incomplete_dist_inc_1d_distinct.select(fueleconomy_dims[:1]).exceptAll(df_fueleconomy_incomplete_reference_1d_distinct.select(fueleconomy_dims[:1])).head(1)) == 0)
display(len(df_fueleconomy_incomplete_reference_1d_distinct.select(fueleconomy_dims[:1]).exceptAll(df_fueleconomy_incomplete_dist_inc_1d_distinct.select(fueleconomy_dims[:1])).head(1)) == 0)
display(len(df_fueleconomy_incomplete_dist_inc_2d_distinct.select(fueleconomy_dims[:2]).exceptAll(df_fueleconomy_incomplete_reference_2d_distinct.select(fueleconomy_dims[:2])).head(1)) == 0)
display(len(df_fueleconomy_incomplete_reference_2d_distinct.select(fueleconomy_dims[:2]).exceptAll(df_fueleconomy_incomplete_dist_inc_2d_distinct.select(fueleconomy_dims[:2])).head(1)) == 0)
display(len(df_fueleconomy_incomplete_dist_inc_6d_distinct.select(fueleconomy_dims[:6]).exceptAll(df_fueleconomy_incomplete_reference_6d_distinct.select(fueleconomy_dims[:6])).head(1)) == 0)
display(len(df_fueleconomy_incomplete_reference_6d_distinct.select(fueleconomy_dims[:6]).exceptAll(df_fueleconomy_incomplete_dist_inc_6d_distinct.select(fueleconomy_dims[:6])).head(1)) == 0)

display(len(df_coil2000_incomplete_dist_inc_1d_distinct.select(coil2000_dims[:1]).exceptAll(df_coil2000_incomplete_reference_1d_distinct.select(coil2000_dims[:1])).head(1)) == 0)
display(len(df_coil2000_incomplete_reference_1d_distinct.select(coil2000_dims[:1]).exceptAll(df_coil2000_incomplete_dist_inc_1d_distinct.select(coil2000_dims[:1])).head(1)) == 0)
display(len(df_coil2000_incomplete_dist_inc_2d_distinct.select(coil2000_dims[:2]).exceptAll(df_coil2000_incomplete_reference_2d_distinct.select(coil2000_dims[:2])).head(1)) == 0)
display(len(df_coil2000_incomplete_reference_2d_distinct.select(coil2000_dims[:2]).exceptAll(df_coil2000_incomplete_dist_inc_2d_distinct.select(coil2000_dims[:2])).head(1)) == 0)
display(len(df_coil2000_incomplete_dist_inc_6d_distinct.select(coil2000_dims[:6]).exceptAll(df_coil2000_incomplete_reference_6d_distinct.select(coil2000_dims[:6])).head(1)) == 0)
display(len(df_coil2000_incomplete_reference_6d_distinct.select(coil2000_dims[:6]).exceptAll(df_coil2000_incomplete_dist_inc_6d_distinct.select(coil2000_dims[:6])).head(1)) == 0)

display(len(df_nba_incomplete_dist_inc_1d_distinct.select(nba_dims[:1]).exceptAll(df_nba_incomplete_reference_1d_distinct.select(nba_dims[:1])).head(1)) == 0)
display(len(df_nba_incomplete_reference_1d_distinct.select(nba_dims[:1]).exceptAll(df_nba_incomplete_dist_inc_1d_distinct.select(nba_dims[:1])).head(1)) == 0)
display(len(df_nba_incomplete_dist_inc_2d_distinct.select(nba_dims[:2]).exceptAll(df_nba_incomplete_reference_2d_distinct.select(nba_dims[:2])).head(1)) == 0)
display(len(df_nba_incomplete_reference_2d_distinct.select(nba_dims[:2]).exceptAll(df_nba_incomplete_dist_inc_2d_distinct.select(nba_dims[:2])).head(1)) == 0)
display(len(df_nba_incomplete_dist_inc_6d_distinct.select(nba_dims[:6]).exceptAll(df_nba_incomplete_reference_6d_distinct.select(nba_dims[:6])).head(1)) == 0)
display(len(df_nba_incomplete_reference_6d_distinct.select(nba_dims[:6]).exceptAll(df_nba_incomplete_dist_inc_6d_distinct.select(nba_dims[:6])).head(1)) == 0)

## Postprocessing

In this section, we do postprocessing to make evaluating the results of the testing easier. This involves the following step(s):
* Renaming all CSV files from their respective "partitioning" naming to proper naming based on the name of the containing folder.

In [ ]:
from pathlib import Path

path = "output"
files = Path(path).rglob('*.csv')

for file in files:
    parent = file.parent.name
    file.rename(Path(file.parent,f"{parent}{file.suffix}"))